In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install plotly
# !pip install --upgrade nbformat
# !pip install nltk
# !pip install spacy # spaCy is an open-source software library for advanced natural language processing
# !pip install WordCloud
# !pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing
!pip install jupyterthemes
!pip install hd5y
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 
# setting the style of the notebook to be monokai theme  
# this line of code is important to ensure that we are able to see the x and y axes clearly
# If you don't run this code line, you will notice that the xlabel and ylabel on any plot is black on black and it will be hard to see them. 


     |████████████████████████████████| 7.0MB 4.3MB/s 
     |████████████████████████████████| 9.5MB 32.2MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
ERROR: google-colab 1.0.0 has requirement notebook~=5.3.0; python_version >= "3.0", but you'll have notebook 6.1.4 which is incompatible.
  Found existing installation: notebook 5.3.1
    Uninstalling notebook-5.3.1:
      Successfully uninstalled notebook-5.3.1
ERROR: Could not find a version that satisfies the requirement hd5y (from versions: none)
ERROR: No matching distribution found for hd5y
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# load the data
df_true = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/True.csv")
df_fake = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/Fake.csv")

df_train = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/test.csv")

df_fnn_train = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/fnn_train.csv")
df_fnn_test = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/fnn_test.csv")
df_fnn_val = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/fnn_dev.csv")




In [ ]:
#clean dataframe df_fnn_train
df_fnn_train.drop(columns=['id'], inplace=True)
df_fnn_train.drop(columns=['date'], inplace=True)
df_fnn_train.drop(columns=['sources'], inplace=True)
df_fnn_train.drop(columns=['paragraph_based_content'], inplace=True)

df_fnn_train['text'] = df_fnn_train['fullText_based_content']
df_fnn_train.drop(columns=['fullText_based_content'], inplace=True)

df_fnn_train.drop(columns=['speaker'], inplace=True)

for i in range(0,len(df_fnn_train)): 
  if df_fnn_train['label_fnn'][i] == 'real':
    df_fnn_train['label_fnn'][i] = 0
  elif df_fnn_train['label_fnn'][i] == 'fake':
    df_fnn_train['label_fnn'][i] = 1

pd.to_numeric(df_fnn_train['label_fnn'],errors='coerce')

df_fnn_train['isfake'] = df_fnn_train['label_fnn']
df_fnn_train.drop(columns=['label_fnn'], inplace=True)

df_fnn_train['title'] = df_fnn_train['statement']
df_fnn_train.drop(columns=['statement'], inplace=True)

In [ ]:
#clean dataframe df_fnn_test

df_fnn_test.drop(columns=['id'], inplace=True)
df_fnn_test.drop(columns=['date'], inplace=True)
df_fnn_test.drop(columns=['sources'], inplace=True)
df_fnn_test.drop(columns=['paragraph_based_content'], inplace=True)

df_fnn_test['text'] = df_fnn_test['fullText_based_content']
df_fnn_test.drop(columns=['fullText_based_content'], inplace=True)

df_fnn_test.drop(columns=['speaker'], inplace=True)

for i in range(0,len(df_fnn_test)): 
  if df_fnn_test['label_fnn'][i] == 'real':
    df_fnn_test['label_fnn'][i] = 0
  elif df_fnn_test['label_fnn'][i] == 'fake':
    df_fnn_test['label_fnn'][i] = 1

pd.to_numeric(df_fnn_test['label_fnn'],errors='coerce')

df_fnn_test['isfake'] = df_fnn_test['label_fnn']
df_fnn_test.drop(columns=['label_fnn'], inplace=True)

df_fnn_test['title'] = df_fnn_test['statement']
df_fnn_test.drop(columns=['statement'], inplace=True)

In [ ]:
#clean dataframe df_fnn_val

df_fnn_val.drop(columns=['id'], inplace=True)
df_fnn_val.drop(columns=['date'], inplace=True)
df_fnn_val.drop(columns=['sources'], inplace=True)
df_fnn_val.drop(columns=['paragraph_based_content'], inplace=True)

df_fnn_val['text'] = df_fnn_val['fullText_based_content']
df_fnn_val.drop(columns=['fullText_based_content'], inplace=True)

df_fnn_val.drop(columns=['speaker'], inplace=True)

for i in range(0,len(df_fnn_val)): 
  if df_fnn_val['label_fnn'][i] == 'real':
    df_fnn_val['label_fnn'][i] = 0
  elif df_fnn_val['label_fnn'][i] == 'fake':
    df_fnn_val['label_fnn'][i] = 1

pd.to_numeric(df_fnn_val['label_fnn'],errors='coerce')

df_fnn_val['isfake'] = df_fnn_val['label_fnn']
df_fnn_val.drop(columns=['label_fnn'], inplace=True)

df_fnn_val['title'] = df_fnn_val['statement']
df_fnn_val.drop(columns=['statement'], inplace=True)

In [ ]:
df_use_to_test = pd.concat([df_fnn_train, df_fnn_test, df_fnn_val]).reset_index(drop = True)

In [ ]:
#clean data use to train df_true and df_fake

# add a target class column to indicate whether the news is real or fake
df_true['isfake'] = 1
df_fake['isfake'] = 0

# Concatenate Real and Fake News
df = pd.concat([df_true, df_fake]).reset_index(drop = True)

#drop columns
df.drop(columns = ['date'], inplace = True)
df.drop(columns = ['subject'], inplace = True)

In [ ]:
#clean data use to train df_train and df_test

# Concatenate Real and Fake News
df2=pd.concat([df_train, df_test]).reset_index(drop = True)

#drop columns
df2.drop(columns=['author'],inplace = True)
df2.drop(columns=['id'], inplace=True)

df2['isfake'] = df2['label']
df2.drop(columns=['label'], inplace=True)

pd.to_numeric(df2['isfake'],errors='coerce')

df2 = df2.dropna(axis=0, subset=['isfake'])
df2 = df2.dropna(axis=0, subset=['text'])
df2 = df2.dropna(axis=0, subset=['title'])


In [ ]:
#concat 2 datafram df and df2
df = pd.concat([df, df2]).reset_index(drop = True)


In [ ]:
df

,title,text,isfake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1.0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1.0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1.0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1.0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1.0
...,...,...,...
65096,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,0.0
65097,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,0.0
65098,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,0.0
65099,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",1.0


In [ ]:
# Obtain additional stopwords from nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [ ]:
# Apply the function to the dataframe
df['clean'] = df['title'].apply(preprocess)

In [ ]:
# Show original news
df['title'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script'

In [ ]:
# Show cleaned up news after removing stopwords
print(df['clean'][0])

['budget', 'fight', 'looms', 'republicans', 'flip', 'fiscal', 'script']


In [ ]:
df

,title,text,isfake,clean
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1.0,"[budget, fight, looms, republicans, flip, fisc..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1.0,"[military, accept, transgender, recruits, mond..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1.0,"[senior, republican, senator, mueller]"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1.0,"[russia, probe, helped, australian, diplomat]"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1.0,"[trump, wants, postal, service, charge, amazon..."
...,...,...,...,...
65096,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,0.0,"[rapper, trump, poster, child, white, supremacy]"
65097,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,0.0,"[playoffs, schedule, matchups, odds, york, times]"
65098,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,0.0,"[macy, said, receive, takeover, approach, huds..."
65099,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",1.0,"[nato, russia, hold, parallel, exercises, balk..."


In [ ]:
# Obtain the total words present in the dataset
list_of_words = []
for i in df.clean:
    for j in i:
        list_of_words.append(j)


In [ ]:
len(list_of_words)

513872

In [ ]:
# Obtain the total number of unique words
total_words = len(list(set(list_of_words)))
total_words

27892

In [ ]:
# join the words into a string
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

In [ ]:
# plot the number of samples in 'subject'
# plt.figure(figsize = (8, 8))
# sns.countplot(y = "subject", data = df)

In [ ]:
# plot the word cloud for text that is Real
# plt.figure(figsize = (20,20)) 
# wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop_words).generate(" ".join(df[df.isfake == 1].clean_joined))
# plt.imshow(wc, interpolation = 'bilinear')

In [ ]:
# plot the word cloud for text that is Fake
# plt.figure(figsize = (20,20)) 
# wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop_words).generate(" ".join(df[df.isfake == 0].clean_joined))
# plt.imshow(wc, interpolation = 'bilinear')

In [ ]:
# length of maximum document will be needed to create word embeddings 
maxlen = -1
for doc in df.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is =", maxlen)

The maximum number of words in any document is = 41


In [ ]:
# visualize the distribution of number of words in a text
# import plotly.express as px
# fig = px.histogram(x = [len(nltk.word_tokenize(x)) for x in df.clean_joined], nbins = 100)
# fig.show()

In [ ]:
from nltk import word_tokenize

In [ ]:
x_train = df['clean_joined']
y_train = np.asarray(df['isfake'])
x_test = df_use_to_test['title']
y_test = df_use_to_test['isfake']

In [ ]:
# Create a tokenizer to tokenize the words and create sequences of tokenized words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)

tokenizer.fit_on_texts(x_test)
test_sequences = tokenizer.texts_to_sequences(x_test)


In [ ]:
print("The encoding for document\n",df.clean_joined[0],"\n is : ",train_sequences[0])

The encoding for document
 budget fight looms republicans flip fiscal script 
 is :  [188, 100, 1775, 35, 2978, 2525, 4135]


In [ ]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
padded_train = pad_sequences(train_sequences,maxlen = 41, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 41, padding = 'post')

In [ ]:
# Sequential Model
model = Sequential()

# embeddidng layer
model.add(Embedding(total_words, output_dim = 128))
# model.add(Embedding(total_words, output_dim = 240))


# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))

# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         3570176   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 3,866,369
Trainable params: 3,866,369
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
filepath="/content/drive/My Drive/fnd-usingML/checkpoints/title-only/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor = 'val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             mode = 'max')

callbacks_list = [checkpoint]

In [ ]:
# train the model
model.fit(padded_train, y_train, batch_size = 64, validation_split = 0.1, epochs = 2, callbacks = callbacks_list)

Epoch 1/2
916/916 [==============================] - ETA: 0s - loss: 0.2737 - acc: 0.8856
Epoch 00001: val_acc improved from 0.83720 to 0.85548, saving model to /content/drive/My Drive/fnd-usingML/checkpoints/title-only/weights-improvement-01-0.86.hdf5
916/916 [==============================] - 40s 43ms/step - loss: 0.2737 - acc: 0.8856 - val_loss: 0.3731 - val_acc: 0.8555
Epoch 2/2
915/916 [============================>.] - ETA: 0s - loss: 0.1575 - acc: 0.9405
Epoch 00002: val_acc did not improve from 0.85548
916/916 [==============================] - 39s 42ms/step - loss: 0.1575 - acc: 0.9405 - val_loss: 0.4143 - val_acc: 0.8338


In [ ]:
# make prediction
pred = model.predict(padded_test)
print(pred)

[[0.4552691 ]
 [0.01792111]
 [0.01151633]
 ...
 [0.00505076]
 [0.9594134 ]
 [0.00324415]]


In [ ]:
# if the predicted value is >0.5 it is real else it is fake
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [ ]:
# getting the accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)

Model Accuracy :  0.5347494804894943


In [ ]:
# get the confusion matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(list(y_test), prediction)
# plt.figure(figsize = (25, 25))
# sns.heatmap(cm, annot = True)

In [ ]:
# category dict
category = { 0: 'Fake News', 1 : "Real News"}

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

In [ ]:
text_test = 'The claim: Trump said, "Some Africans are lazy fools only good at eating, lovemaking and stealing" Falsely attributed quotes on social media that target politicians have become a popular form of misinformation, particularly as the 2020 election season enters its final stretch. A phrase claiming to come from President Donald Trump that circulated in October 2015 is making the rounds again on social media platforms. The article, allegedly published Oct. 27, 2015, claims Trump, while in Indianapolis, said African Americans are "lazy." The story does not reference a date or context for Trump speaking in that city. The social media post contains what appears to be a picture of the print version of the article, which has "Africans Are Lazy, Good At Sex, Theft", as a headline, above a photo of Trump. The article quotes Trump as referring to Africans and African Americans interchangeably. '
print(text_test)

In [ ]:
corpus = []

new = re.sub('[^a-zA-Z]', ' ', text_test)
new = new.lower()
new = new.split()
lemmatizer = WordNetLemmatizer() 
sw = stopwords.words('english')
new = [lemmatizer.lemmatize(word) for word in new if not word in set(sw)]
new = ' '.join(new)
corpus.append(new)

text_sequences_test = tokenizer.texts_to_sequences(corpus)
padded_text_test = pad_sequences(text_sequences_test,maxlen = 40, truncating = 'post')
print(padded_text_test)


In [ ]:
prediction_text_test = model.predict(padded_text_test)
result = ''
# for i in range(len(prediction_text_test)):
if prediction_text_test[0] > 0.5:
    result = 'fake'
else:
    result = 'true'

print(result)
print(prediction_text_test)
